# Spatial analysis of Epithelial (binarised marker) vs Fibroblast

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os, sys

other_dir_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.append(other_dir_path)

print(other_dir_path)
import IMCprocess.spatial_analysis.spatial_analysis as sa
import IMCprocess.spatial_analysis.combi_enrich as ce
import IMCprocess.utils.utils as ut

# I have to use custom Img_anndata code
import Img_anndata as utan

/Users/pattama/Desktop/UiB/cancer/git_repo/IMCprocess


## Import mask and level1 annotated object and create epithelial object

In [2]:
data_dir = '/Users/pattama/Desktop/UiB/cancer/example_share_code/share_with_vladan/mask_analysis_full_pipeline/example_data/'
input_mask = data_dir + 'Patient2_001.tiff'
output_dir = 'output'
level1_file = data_dir + '00hanna_adata_totalNB_obj_Level1.pkl'
myimg_id = 'R2_001'
Img_obj_level1 = ut.read_pickle_obj(level1_file)
# Get adata level1 for img_id R2_001
Img_level1_R2_001 = ut.filter_adata_obs(Img_obj_level1.adata_dict['level1'],
                                        'img_id', 
                                        [myimg_id])
# Get Epi adata
Epi_adata = ut.filter_adata_obs(Img_level1_R2_001,
                                'level1_annotated',
                                ['Ep'])

## Label cells for neg or pos cells from chosen threshold and add to obs in object

In [3]:
def binary_df(df, threshold):
    df_binary = df.applymap(lambda x: 'pos' if x >= threshold else 'neg')
    df_binary.columns = [col + '_'+ str(round(threshold,2)) for col in df.columns]
    return(df_binary)

In [4]:
# User provide input marker column and 
# selected threshold for each marker 
# (User can choose more than one threshold 
# for the same marker to compare)
marker_threshold_dict = {'168Er_Ki67': [0.2],
                         '143Nd_Vimentin': [0.2, 0.3],
                         '169Tm_CD24': [0.4]
}

In [5]:
binary_df_ls = []
for marker, threshold_list in marker_threshold_dict.items():
    marker_bi_ls = [binary_df(Epi_adata.to_df()[[marker]], threshold) 
                    for threshold in threshold_list]
    marker_bi_df = pd.concat(marker_bi_ls, axis=1)
    binary_df_ls.append(marker_bi_df)
binary_all_marker = pd.concat(binary_df_ls, axis=1)

In [6]:
binary_all_marker.head()

,168Er_Ki67_0.2,143Nd_Vimentin_0.2,143Nd_Vimentin_0.3,169Tm_CD24_0.4
Cell_id,,,,
14301_3,pos,pos,pos,pos
14304_3,pos,pos,pos,pos
14305_3,neg,pos,pos,neg
14307_3,neg,pos,pos,pos
14308_3,neg,neg,neg,pos


In [7]:
# add binary df in adata obs
Epi_adata.obs = pd.concat([Epi_adata.obs, binary_all_marker], axis=1)

## Calculate spatial analysis between Epithelial (binarise) and Fibroblast

In [19]:
# import Fibro adata
Fibro_obj = ut.read_pickle_obj(data_dir + '02hanna_adata_totalNB_obj_Level2.pkl')
# Get adata Fibro only for img_id R2_001
Fibro_adata = ut.filter_adata_obs(Fibro_obj.adata_dict['Fibro_level2'],
                                  'img_id', 
                                  [myimg_id])

In [20]:
Fibro_adata.obs

,img_id,PhenoGraph_clusters,level1_annotated,Fibro_level2_annotated
Cell_id,,,,
14302_3,R2_001,11,Fibro,aSMA+_FAPlow_PDGFRalow_PDGFRblow
14303_3,R2_001,24,Fibro,aSMA+_PDGFRalow_PDGFRblow
14306_3,R2_001,15,Fibro,aSMAlow_FAP+_PDGFRalow_PDGFRblow
14309_3,R2_001,13,Fibro,aSMA+_FAPlow_PDGFRalow_PDGFRblow
14310_3,R2_001,20,Fibro,aSMA+_FAP+_PDGFRalow_PDGFRblow
...,...,...,...,...
21438_3,R2_001,0,Fibro,Neg
21450_3,R2_001,9,Fibro,FAP+_PDGFRalow_PDGFRblow
21451_3,R2_001,15,Fibro,aSMAlow_FAP+_PDGFRalow_PDGFRblow
